In [2]:
from platform import python_version

print(python_version())

# %pip install langchain
# %pip install -U langchain-community
# %pip install pypdf
# %pip install spacy
# !python3.9 -m pip install spacy-llm
# !python3.9 -m spacy download en_core_web_sm
# %pip install scikit-learn
# %pip install -U sentence-transformers
# %pip install keybert
# %pip install transformers sentence-transformers nltk


3.9.21


#### Document reading and split

In [3]:
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
import os

# Directory containing your PDF files
directory_path = 'TCs'

# Initialize PyPDFLoader for each PDF in the directory
loaders = [PyPDFLoader(os.path.join(directory_path, f)) for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Load documents from PDFs 
tc_docs = []
for loader in loaders:
    tc_docs.extend(loader.load())


tc_data = [
    Document(
        page_content=doc.page_content, 
        metadata={
            "source": doc.metadata['source'].removeprefix('TCs'),
        }
    )
    for doc in tc_docs
]

apple_docs = [doc for doc in tc_data if "apple_macos_english" in doc.metadata["source"].lower()]

print(f"Number of Apple-related documents: {len(apple_docs)}")

page_contents_apple = [doc.page_content for doc in apple_docs]

combined_apple = "".join(page_contents_apple) 


Number of Apple-related documents: 17


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=60
)
# split_docs = text_splitter.split_text(data)
split_docs_apple = text_splitter.split_text(combined_apple)
print(len(split_docs_apple))

# function for functionality of pipeline
def split_text(combined_text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=60
    )
    return text_splitter.split_text(combined_text)

150


### Entity & Relationships

In [5]:
import os
import json
import spacy
from collections import Counter
from pathlib import Path
from wasabi import msg
from spacy_llm.util import assemble

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# traditional spacy NER
# def split_document_sent(text):
#     nlp = spacy.load("en_core_web_sm")
#     doc = nlp(text)
#     return [sent.text.strip() for sent in doc.sents] # referencial

def summarize_section(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    if not sentences:
        return "No summary available"

    try:
        # TF-IDF (sentence embedding)
        vectorizer = TfidfVectorizer(stop_words='english')
        sentence_vectors = vectorizer.fit_transform(sentences).toarray()

        avg_vector = np.mean(sentence_vectors, axis=0).reshape(1, -1)
        similarities = cosine_similarity(avg_vector, sentence_vectors)

        # most relevant
        most_relevant_index = np.argmax(similarities)
        return sentences[most_relevant_index]

    # in case there is empty input
    except ValueError:
        return "No summary available"

# spacy-llm relationship extraction
# def process_text(nlp, text, verbose=False):
#     doc = nlp(text)
#     if verbose:
#         msg.text(f"Text: {doc.text}")
#         msg.text(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")
#         msg.text("Relations:")
#         for r in doc._.rel:
#             msg.text(f"  - {doc.ents[r.dep]} [{r.relation}] {doc.ents[r.dest]}")
#     return doc

def run_pipeline(combined, config_path, filename, examples_path=None, verbose=False):
    if not os.getenv("OPENAI_API_KEY"):
        msg.fail("OPENAI_API_KEY env variable was not found. Set it and try again.", exits=1)

    sections = split_text(combined)
    nlp = assemble(config_path, overrides={} if examples_path is None else {"paths.examples": str(examples_path)})

    # Initialize counters and storage
    processed_data = []
    entity_counts = Counter()
    relation_counts = Counter()

    for section in sections:
        summary = summarize_section(section)
        doc = nlp(summary)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        relations = [(doc.ents[r.dep].text, r.relation, doc.ents[r.dest].text) for r in doc._.rel]

        # processed data
        processed_data.append({
            'original_text': section,
            'summary': summary,
            'entities': entities,
            'relations': relations})

        entity_counts.update([ent[1] for ent in entities])
        relation_counts.update([rel[1] for rel in relations])

    with open(filename, 'w') as f:
        json.dump(processed_data, f)

    msg.text(f"Entity counts: {entity_counts}")
    msg.text(f"Relation counts: {relation_counts}")

config_path = Path("config.cfg")
examples_path = None
verbose = True

file_apple = run_pipeline(combined_apple, config_path, 'apple_processed_data.json', None, verbose)

/usr/local/lib/python3.9/site-packages/spacy_llm/pipeline/llm.py:143: UserWarning: Task supports sharding, but model does not provide context length. Data won't be sharded, prompt might exceed the model's context length. Set context length in your config. If you think spacy-llm should provide the context length for this model automatically, report this to https://github.com/explosion/spacy-llm/issues.
  warnings.warn(


Entity counts: Counter({'ORG': 263, 'GPE': 81, 'CARDINAL': 20, 'PERSON': 20,
'ORDINAL': 17, 'LAW': 13, 'WORK_OF_ART': 10, 'PRODUCT': 1, 'DATE': 1, 'EVENT':
1, 'TIME': 1})
Relation counts: Counter({'is located in': 43, 'is a country in': 27, 'uses':
20, 'related_to': 20, 'provides': 16, 'includes': 10, 'runs_on': 9, 'is_a': 7,
'refers_to': 6, 'affiliated_with': 5, 'has access to': 5, 'supports': 4,
'related to': 3, 'licensed_by': 3, 'runs': 3, 'included_with': 3,
'associated_with': 3, 'governs': 2, 'affects': 2, 'is part of': 2, 'owned_by':
2, 'does not endorse': 2, 'is a type of': 2, 'maintains': 2, 'owns': 1,
'accompanied by': 1, 'competes_with': 1, 'obtained_from': 1, 'subject to': 1,
'set forth in': 1, 'produces': 1, 'ownership': 1, 'supported_on': 1,
'branded_by': 1, 'downloads': 1, 'delivers': 1, 'is_related_to': 1, 'enables':
1, 'authorizes': 1, 'involves': 1, 'governed_by': 1, 'has_rights_over': 1,
'license_for': 1, 'controls': 1, 'prohibits': 1, 'restricts': 1, 'transfers
softw

### Addition of description per chunk of text as well as simplification 

#### Add description

In [6]:
def summarize_section(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    vectorizer = TfidfVectorizer(stop_words='english')
    # convert to array in order to perform cosine similarity
    sentence_vectors = vectorizer.fit_transform(sentences).toarray()

    avg_vector = np.mean(sentence_vectors, axis=0)
    avg_vector = avg_vector.reshape(1, -1)
    similarities = cosine_similarity(avg_vector, sentence_vectors)

    # index with highest cosine similarity to the average vector
    most_relevant_index = np.argmax(similarities)

    return sentences[most_relevant_index]


print(summarize_section(split_docs_apple[1]))
split_docs_apple[1]

AT https://www.apple.com/legal/sales-support/. YOU MUST RETURN 
THE ENTIRE HARDWARE/SOFTWARE PACKAGE IN ORDER TO OBTAIN A REFUND.


'ACQUIRED THE APPLE SOFTWARE AS PART OF AN APPLE HARDWARE PURCHASE AND IF YOU \nDO NOT AGREE TO THE TERMS OF THIS LICENSE, YOU MAY RETURN THE ENTIRE APPLE \nHARDWARE/SOFTWARE PACKAGE WITHIN THE RETURN PERIOD TO THE APPLE STORE OR \nAUTHORIZED DISTRIBUTOR WHERE YOU OBTAINED IT FOR A REFUND, SUBJECT TO APPLE’S \nRETURN POLICY FOUND AT https://www.apple.com/legal/sales-support/. YOU MUST RETURN \nTHE ENTIRE HARDWARE/SOFTWARE PACKAGE IN ORDER TO OBTAIN A REFUND.'

#### Simplification with Lexical Simplification with BERT

In [8]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

nltk.download('punkt')

# Load models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", trust_remote_code=True)
similarity_model = SentenceTransformer("all-MiniLM-L6-v2")

def mask_word(sentence, word):
    """Replace a word in the sentence with [MASK]."""
    return sentence.replace(word, "[MASK]")

def generate_candidates(masked_sentence, top_k=5):
    """Generate replacement candidates using MLM."""
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits

    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    mask_logits = logits[0, mask_token_index, :]
    top_tokens = mask_logits.topk(top_k, dim=-1).indices[0].tolist()

    candidates = [tokenizer.decode([token]).strip() for token in top_tokens]
    return candidates

def rank_candidates(sentence, word, candidates):
    """Rank candidates based on semantic similarity and simplicity."""
    original_embedding = similarity_model.encode(sentence)
    rankings = []

    for candidate in candidates:
        replaced_sentence = sentence.replace(word, candidate)
        candidate_embedding = similarity_model.encode(replaced_sentence)
        similarity = util.cos_sim(original_embedding, candidate_embedding)[0][0].item()
        rankings.append((candidate, similarity))

    # Sort by similarity and word length (shorter words preferred for simplicity)
    rankings.sort(key=lambda x: (-x[1], len(x[0])))
    return [candidate for candidate, _ in rankings]

def simplify_text(text, complexity_threshold=5):
    """Simplify text by replacing complex words."""
    simplified_sentences = []
    sentences = sent_tokenize(text)

    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            # Detect complex words by length
            if len(word) > complexity_threshold:  
                masked_sentence = mask_word(sentence, word)
                candidates = generate_candidates(masked_sentence)
                ranked_candidates = rank_candidates(sentence, word, candidates)

                if ranked_candidates:
                    sentence = sentence.replace(word, ranked_candidates[0], 1)
        simplified_sentences.append(sentence)

    return " ".join(simplified_sentences)


text = """
The user shall be deemed to have consented to the terms and conditions herein referred to as the agreement. 
You agree to be bound by the obligations under this contract.
"""

simplified_text = simplify_text(text)
print("Simplified Text:", simplified_text)


[nltk_data] Downloading package punkt to /Users/meryjoy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert

Simplified Text: 
The user shall be deemed to have consent to the terms and conditions , referred to as the agreement. You agree to be bound by the contract under this contract.


Still work ahead for this part of the project, might be for later, not enough time

### Create Cypher Queries with eFLINT notation

In [9]:
def classify_and_generate_queries(json_data, file_path):
    nodes = {}
    relationships = []

    # Enhanced mapping function for entity types based on eFLINT elements
    def map_eflint_type(entity_type, entity_name):
        if 'http://' in entity_type or 'https://' in entity_name:
            return 'FACT'
        if entity_type in ['CARDINAL', 'ORDINAL']:
            return 'NUMBER'
        if entity_type == 'ORG':
            if 'terms' in entity_name.lower() or 'conditions' in entity_name.lower() or 'agreement' in entity_name.lower():
                return 'DUTY'
            else:
                return 'ACTOR'
        if 'section' in entity_name.lower() or 'section' in entity_type.lower():
            return 'SECTION'
        mapping = {
            'PERSON': 'ACTOR',
            'EVENT': 'EVENT',
            'LAW': 'DUTY',
            'WORK_OF_ART': 'ACT',
            'CONDITION': 'CONDITION',
            'DATE': 'DATE',
        }
        return mapping.get(entity_type, 'FACT')

    # Process summaries and associate them with entities
    for item in json_data:
        summary = item.get('summary', 'No summary available')
        entities = item.get('entities', [])
        relations = item.get('relations', [])

        for entity in entities:
            entity_name, entity_type = entity[:2]
            mapped_type = map_eflint_type(entity_type, entity_name)
            node_id = f"{entity_name.replace(' ', '_')}_{mapped_type}"
            if node_id not in nodes:
                nodes[node_id] = {
                    'name': entity_name,
                    'type': mapped_type,
                    'descriptions': [summary],  # Start with the current summary
                }
            else:
                nodes[node_id]['descriptions'].append(summary)  # Append additional summaries

        # Add relationships if present
        for relation in relations:
            src_id = f"{relation[0].replace(' ', '_')}_{map_eflint_type(entity_type, relation[0])}"
            tgt_id = f"{relation[2].replace(' ', '_')}_{map_eflint_type(entity_type, relation[2])}"
            relationship_type = relation[1].replace(' ', '_').replace('-', '_')
            relationships.append((src_id, relationship_type, tgt_id))

    # Create Cypher queries for nodes
    node_queries = [
        f"""
        MERGE (n:{data['type']} {{name: '{data['name'].replace("'", "")}'}})
        SET n.id = '{node_id.replace("'", "")}', 
        n.descriptions = {json.dumps([desc.replace("'", "") for desc in data['descriptions']])}
        """
        for node_id, data in nodes.items() if data['type'] != 'NUMBER'
    ]

    # Create Cypher queries for relationships
    relationship_queries = [
        f"""
        MATCH (a), (b)
        WHERE a.id = '{rel[0].replace("'", "")}' AND b.id = '{rel[2].replace("'", "")}'
        MERGE (a)-[:{rel[1].replace("'", "")}]->(b)
        """
        for rel in relationships
    ]

    queries = node_queries + relationship_queries

    # Save queries to a text file
    with open(file_path, 'w') as file:
        for query in queries:
            file.write(query.strip() + '\n')

    return queries


In [10]:
with open('apple_processed_data.json', 'r') as file:
    json_data = json.load(file)

queries_apple = classify_and_generate_queries(json_data, 'apple_cypher_queries.txt')

In [11]:
from neo4j import GraphDatabase

def execute_queries(queries, uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        for query in queries:
            session.run(query)
    driver.close()

uri = "neo4j://localhost:7687"
user = "neo4j"
password = "movies11"

execute_queries(queries_apple, uri, user, password)

## Other option: 
### Using Langchain for Queries

In [13]:
# %pip install langchain_openai
# %pip install langchain_experimental
# %pip install neo4j langchain-neo4j
# %pip install yfiles_jupyter_graphs

In [14]:
if not os.getenv("NEO4J_URI"):
    print("nope")
if not os.getenv("NEO4J_USERNAME"):
    print("nope")
if not os.getenv("NEO4J_PASSWORD"):
    print("nope")

In [15]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
tc_document = text_splitter.split_documents(tc_data)

In [16]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
     
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(tc_document)

In [17]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph()
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [18]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 80"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [19]:
# showGraph()

## Summarization based RAG

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI

# Neo4j Connection
class KnowledgeGraph:
    def __init__(self, uri, user, password):
        from neo4j import GraphDatabase
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def get_all_summaries(self):
        with self.driver.session() as session:
            query = """
            MATCH (n)
            RETURN DISTINCT n.descriptions AS descriptions
            """
            results = session.run(query)
            summaries = []
            for record in results:
                descriptions = record["descriptions"]
                if descriptions:
                    summaries.extend(descriptions)
            return list(set(summaries))

#### Summary in Paragraph

In [13]:
# Simplified Summarization Workflow
def summarize_in_chunks(summaries, llm):
    # summaries to single text
    combined_text = " ".join(summaries)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    split_texts = text_splitter.split_text(combined_text)

    # summarize each split / chunk
    chunk_prompt = PromptTemplate.from_template(
        """Summarize the following text while preserving key relationships, events, and timelines:
        {text}
        Helpful Answer:"""
    )
    llm_chain = LLMChain(llm=llm, prompt=chunk_prompt)

    chunk_summaries = []
    for chunk in split_texts:
        summary = llm_chain.run({"text": chunk})
        chunk_summaries.append(summary)

    # combination into final summary
    final_prompt = PromptTemplate.from_template(
        """Combine the following summaries into a cohesive final summary:
        {text}
        Helpful Answer:"""
    )
    final_chain = LLMChain(llm=llm, prompt=final_prompt)
    final_summary = final_chain.run({"text": " ".join(chunk_summaries)})

    return final_summary

In [105]:
# def clear_graph(uri, user, password):
#     driver = GraphDatabase.driver(uri, auth=(user, password))
#     with driver.session() as session:
#         session.run("MATCH (n) DETACH DELETE n")
#     driver.close()

# # Usage
# clear_graph("neo4j://localhost:7687", "neo4j", "movies11")


#### Summary in bullet points

In [14]:
# Simplified Summarization Workflow
def summarize_in_points(summaries, llm):
    # summaries to single text
    combined_text = " ".join(summaries)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=70)
    split_texts = text_splitter.split_text(combined_text)

    # summarize each split / chunk
    chunk_prompt = PromptTemplate.from_template(
        """Summarize the following text into a structured format with bullet points and headings, ensuring that key relationships and events are preserved: 
        {text} 
        Helpful Answer:"""
    )
    llm_chain = LLMChain(llm=llm, prompt=chunk_prompt)

    chunk_summaries = []
    for chunk in split_texts:
        summary = llm_chain.run({"text": chunk})
        chunk_summaries.append(summary)

    # combination into final summary
    final_prompt = PromptTemplate.from_template(
        """Combine the following summaries into a structured format with bullet points and headings, while keeping the same structure, into a cohesive final summary: 
        {text} 
        Helpful Answer:"""
    )
    final_chain = LLMChain(llm=llm, prompt=final_prompt)
    final_summary = final_chain.run({"text": " ".join(chunk_summaries)})

    return final_summary

## RAG flow with paragraphs

In [42]:
# RAG Workflow
def rag_summarization_with_chunks(uri, user, password):
    # retrieve descriptions
    kg = KnowledgeGraph(uri, user, password)
    summaries = kg.get_all_summaries()
    kg.close()

    if not summaries:
        print("No summaries found in the knowledge graph.")
        return "No summaries found in the knowledge graph."

    unique_summaries = list(set(summaries))
    # print("Unique Descriptions Retrieved:")
    # for idx, summary in enumerate(unique_summaries, start=1):
    #     print(f"{idx}: {summary}")

    llm = ChatOpenAI(temperature=0, model_name="gpt-4")
    final_summary = summarize_in_chunks(unique_summaries, llm)
    return final_summary

# Perform RAG summarization with chunking
apple_par_summary = rag_summarization_with_chunks(uri, user, password)
print("\nFinal Summary:")
print(apple_par_summary)


Final Summary:
The text outlines the terms and conditions for using Apple Software on a supported Mac Computer, including automatic downloads and installations of software changes. The number of copies that can be downloaded depends on the user's license with Apple. The software is licensed under the MPEG-4 Visual Patent Portfolio License for personal and non-commercial use. It also discusses the use of Gracenote software, licensed by MPEG LA, for personal and non-commercial activities, which enables applications to identify discs and files and obtain music-related information. The text also mentions the use of Setup/Migration Assistant for transferring software between Apple-branded computers. Users are advised to suspend the ability to pay with virtual Supported Cards on their Mac Computer by putting it into Lost Mode. If the license for the Gracenote software terminates, users must cease all use of the Gracenote Data, Software, and Servers. The text also mentions potential risks su

## RAG flow with bullet points

In [21]:
# RAG Workflow
def rag_summarization_with_points(uri, user, password):
    # retrieve descriptions
    kg = KnowledgeGraph(uri, user, password)
    summaries = kg.get_all_summaries()
    kg.close()

    if not summaries:
        print("No summaries found in the knowledge graph.")
        return "No summaries found in the knowledge graph."

    unique_summaries = list(set(summaries))
    # print("Unique Descriptions Retrieved:")
    # for idx, summary in enumerate(unique_summaries, start=1):
    #     print(f"{idx}: {summary}")

    llm = ChatOpenAI(temperature=0, model_name="gpt-4")
    final_summary = summarize_in_points(unique_summaries, llm)
    return final_summary

# Perform RAG summarization with chunking
apple_points_summary = rag_summarization_with_points(uri, user, password)
print("\nFinal Summary:")
print(apple_points_summary)


Final Summary:
- **Interference with Apple's Security Mechanisms**
  - Prohibits circumventing or interfering with Apple's verification, storage, or authentication mechanisms.
  - Includes digital signing, digital rights management, and other security mechanisms implemented in or by Apple software, services, or technology.
  - Prohibits enabling others to interfere with these mechanisms.

- **Apple Software Components and Open Source Programs**
  - Certain components of the Apple Software and third-party open source programs included with the Apple Software may be made available by Apple on its Open Source website.

- **Key Information**
  - The AMR encoding and decoding functionality in the product is not licensed for use in cellular communications infrastructure, including base stations.
  - For Australian consumers, the license does not affect or intend to affect statutory rights under the Australian Consumer Law.

- **Apple Products and Services**
  - Requires operating software t

In [16]:
with open('apple_bullet_points.txt', "w") as file:
        file.write(apple_points_summary)

In [20]:
apple_points_summary2 = rag_summarization_with_points(uri, user, password)
print("\nFinal Summary:")
print(apple_points_summary2)



Final Summary:
- **Interference with Apple's Security Mechanisms**
  - Prohibits circumventing or interfering with Apple's verification, storage, or authentication mechanisms.
  - Includes digital signing, digital rights management, and other security mechanisms implemented in or by Apple software, services, or other Apple technology.
  - Prohibits enabling others to interfere with these mechanisms.

- **Apple Software Components and Open Source Programs**
  - Certain components of the Apple software, and third-party open source programs included with the Apple software, have been or may be made available by Apple on its Open Source.
  - Website: https://www.opensource.apple.com/ (referred to as "Open-Sourced Components")

- **Key Information**
  - The AMR encoding and decoding functionality in the product is not licensed for use in cellular communications infrastructure. This includes base stations.
  - For Australian consumers: The license does not affect or intend to affect statuto

In [19]:
apple_points_summary3 = rag_summarization_with_points(uri, user, password)
print("\nFinal Summary:")
print(apple_points_summary3)


Final Summary:
- **Interference with Apple's Security Mechanisms**
  - Prohibits circumventing or interfering with Apple's verification, storage, or authentication mechanisms.
  - Includes digital signing, digital rights management, and other security mechanisms implemented in or by Apple software, services, or technology.
  - Prohibition extends to enabling others to interfere with these mechanisms.

- **Components of Apple Software**
  - Certain components of the Apple Software, and third-party open source programs included with the Apple Software, may be made available by Apple on its Open Source website.
  - The product's AMR encoding and decoding functionality is not licensed for use in cellular communications infrastructure, including base stations.

- **For Australian consumers**
  - The license does not affect or intend to affect statutory rights under Australian Consumer Law.
  - Includes consumer guarantees.

- **Apple Products and Services**
  - Requires operating software 

In [22]:
with open('apple_bullet_points2.txt', "w") as file:
        file.write(apple_points_summary3)